### This notebook runs the dimensional load for all dims from sybase.

In [2]:
%run SLV_NB_Functions

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 4, Finished, Available, Finished)

### Dim Store Section

In [3]:
ftiendas_path = "abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Tables/ftiendas"
#ftiendas_ext_path = "abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Tables/ftiendas"
distritos_path = "abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Tables/distritos"
regiones_path = "abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Tables/regiones"
zonas_path = "abfss://b2c899fb-e571-4496-aebf-c7a23083635a@onelake.dfs.fabric.microsoft.com/a00cf91a-f92e-498a-9f14-ba10221fb05a/Tables/zonas"

df = spark.read.format("delta").load(ftiendas_path)
#df = spark.read.format("delta").load(ftiendas_path)
df = spark.read.format("delta").load(distritos_path)
df = spark.read.format("delta").load(regiones_path)
df = spark.read.format("delta").load(zonas_path)

spark.read.format("delta").load(ftiendas_path).createOrReplaceTempView("temp_stores")
#spark.read.format("delta").load(ftiendas_path).createOrReplaceTempView("temp_stores_ext")
spark.read.format("delta").load(distritos_path).createOrReplaceTempView("temp_districts")
spark.read.format("delta").load(regiones_path).createOrReplaceTempView("temp_regions")
spark.read.format("delta").load(zonas_path).createOrReplaceTempView("temp_zones")


StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 5, Finished, Available, Finished)

In [4]:
#tiendas_df = spark.read.parquet(get_latest_file('ftiendas'))
tiendas_ext_df = spark.read.parquet(get_latest_file('ftiendas_ext'))
#distritos_df = spark.read.parquet(get_latest_file('distritos'))
#regiones_df = spark.read.parquet(get_latest_file('regiones'))
#zonas_df = spark.read.parquet(get_latest_file('zonas'))

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 6, Finished, Available, Finished)

In [5]:
#tiendas_df.createTempView('temp_stores')
tiendas_ext_df.createTempView('temp_stores_ext')
#distritos_df.createTempView('temp_districts')
#zonas_df.createTempView('temp_zones')
#regiones_df.createTempView('temp_regions')

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 7, Finished, Available, Finished)

In [13]:
%%sql 

select * from temp_regions
order by 1

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 15, Finished, Available, Finished)

<Spark SQL result set with 37 rows and 8 fields>

In [21]:
%%sql
DESCRIBE temp_stores;

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 23, Finished, Available, Finished)

<Spark SQL result set with 110 rows and 3 fields>

In [12]:
%%sql 

select * from temp_zones
order by 1

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 14, Finished, Available, Finished)

<Spark SQL result set with 194 rows and 6 fields>

In [14]:
%%sql 

select * from temp_districts
order by 1

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 16, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 5 fields>

In [15]:
%%sql 

select * from temp_stores_ext
order by 1

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 17, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 23 fields>

In [18]:
tempstore.printSchema()

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 20, Finished, Available, Finished)

NameError: name 'tempstore' is not defined

In [10]:
%%sql 

select * from temp_stores
order by 1

StatementMeta(, e5401fb5-2518-4557-918a-7f8c1891f838, 12, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 110 fields>

In [ ]:
%%sql
SELECT 
            t.TCLAVE AS src_store_id,
            t.TNOMBRE AS store_name,
            t.UDDATE AS update_date_stores,
            t.tcolonia AS neighborhood,
            t.tdelmun AS borough,
            t.testado AS state,
            t.tcp AS zipcode,
            t.tcalle AS street,
            t.c_pais AS country_code,
            t.c_estado AS state_code,
            t.c_municipio AS borough_code,
            t.c_colonia AS neighbor_code,
            t.TTAMANOTIPO AS store_classification,
            d.ddesc AS district_manager,
            d.dclave AS src_district_id,
            z.zona_desc AS zone_manager,
            z.zona_id AS src_zone_id,
            z.supervisor AS zone_supervisor,
            r.rdesc AS region_name,
            r.rclave AS src_region_id,
            CASE 
                WHEN t.TCLAVE BETWEEN 991 AND 999 THEN 'Virtual Store'
                WHEN t.TCLAVE BETWEEN 1000 AND 1099 THEN 'Warehouse'
                ELSE 'Store'
            END AS store_type,
            o.first_sale_or_fallback_date,
            s.sales_days,
            o.days_open,
            CASE 
                WHEN t.TCLAVE BETWEEN 991 AND 999 THEN 'Virtual Store'
                WHEN t.TCLAVE BETWEEN 1000 AND 1099 THEN 'Warehouse'
                WHEN o.days_open >= 366 AND s.sales_days >= 366 THEN 'Full-Year Active Store'
                WHEN o.days_open >= 366 AND s.sales_days < 366 THEN 'Partially Active Same Store'
                ELSE 'New Store'
            END AS store_sales_category,
            CASE t2.int_1 
                WHEN 0 THEN 'Closed' 
                WHEN 1 THEN 'Open' 
                WHEN 2 THEN 'To Be Opened' 
                WHEN 3 THEN 'Closure' 
                WHEN 4 THEN 'Remodeling' 
                ELSE 'Unknown' 
            END AS store_status
        FROM temp_stores t
        LEFT JOIN temp_stores_ext t2 ON t.tclave = t2.tclave
        LEFT JOIN temp_districts d ON d.dclave = t.dclave
        LEFT JOIN temp_zones z ON z.zona_id = d.zona_id
        LEFT JOIN temp_regions r ON r.rclave = z.rclave
        LEFT JOIN cte_sales_day_count s ON s.store_id_from = t.tclave
        LEFT JOIN cte_store_open_days o ON o.store_id = t.tclave

In [5]:
%%sql
MERGE INTO DimStores AS target
USING (
    WITH cte_first_date_sales AS (
        SELECT store_id_from, MIN(movement_date) AS mdate
        FROM fmovarti_intermediate_silver
        WHERE movement_id = 'VT'
        GROUP BY store_id_from
    ),
    cte_sales_day_count AS (
        SELECT store_id_from,
               COUNT(DISTINCT movement_date) AS sales_days
        FROM fmovarti_intermediate_silver
        WHERE movement_id = 'VT'
            AND movement_date >= DATE_SUB(CURRENT_DATE(), 370)
        GROUP BY store_id_from
    ),
    cte_store_open_days AS (
        SELECT 
            t.tclave AS store_id,
            DATEDIFF(CURRENT_DATE(), COALESCE(c.mdate, t2.date_6, t2.date_1, t2.date_2)) AS days_open,
            COALESCE(c.mdate, t2.date_6, t2.date_1, t2.date_2) AS first_sale_or_fallback_date
        FROM temp_stores t
        LEFT JOIN temp_stores_ext t2 ON t.tclave = t2.tclave
        LEFT JOIN cte_first_date_sales c ON c.store_id_from = t.tclave

    ),
    final_data AS (
        SELECT 
            t.TCLAVE AS src_store_id,
            t.TNOMBRE AS store_name,
            t.UDDATE AS update_date_stores,
            t.tcolonia AS neighborhood,
            t.tdelmun AS borough,
            t.testado AS state,
            t.tcp AS zipcode,
            t.tcalle AS street,
            t.c_pais AS country_code,
            t.c_estado AS state_code,
            t.c_municipio AS borough_code,
            t.c_colonia AS neighbor_code,
            t.TTAMANOTIPO AS store_classification,
            d.ddesc AS district_manager,
            d.dclave AS src_district_id,
            z.zona_desc AS zone_manager,
            z.zona_id AS src_zone_id,
            z.supervisor AS zone_supervisor,
            r.rdesc AS region_name,
            r.rclave AS src_region_id,
            CASE 
                WHEN t.TCLAVE BETWEEN 991 AND 999 THEN 'Virtual Store'
                WHEN t.TCLAVE BETWEEN 1000 AND 1099 THEN 'Warehouse'
                ELSE 'Store'
            END AS store_type,
            o.first_sale_or_fallback_date,
            s.sales_days,
            o.days_open,
            CASE 
                WHEN t.TCLAVE BETWEEN 991 AND 999 THEN 'Virtual Store'
                WHEN t.TCLAVE BETWEEN 1000 AND 1099 THEN 'Warehouse'
                WHEN o.days_open >= 366 AND s.sales_days >= 366 THEN 'Full-Year Active Store'
                WHEN o.days_open >= 366 AND s.sales_days < 366 THEN 'Partially Active Same Store'
                ELSE 'New Store'
            END AS store_sales_category,
            CASE t2.int_1 
                WHEN 0 THEN 'Closed' 
                WHEN 1 THEN 'Open' 
                WHEN 2 THEN 'To Be Opened' 
                WHEN 3 THEN 'Closure' 
                WHEN 4 THEN 'Remodeling' 
                ELSE 'Unknown' 
            END AS store_status
        FROM temp_stores t
        LEFT JOIN temp_stores_ext t2 ON t.tclave = t2.tclave
        LEFT JOIN temp_districts d ON d.dclave = t.dclave
        LEFT JOIN temp_zones z ON z.zona_id = d.zona_id
        LEFT JOIN temp_regions r ON r.rclave = z.rclave
        LEFT JOIN cte_sales_day_count s ON s.store_id_from = t.tclave
        LEFT JOIN cte_store_open_days o ON o.store_id = t.tclave
    )

    SELECT * FROM final_data
) AS source
ON target.src_store_id = source.src_store_id

WHEN MATCHED THEN
    UPDATE SET
        target.store_name = source.store_name,
        target.update_date_stores = source.update_date_stores,
        target.neighborhood = source.neighborhood,
        target.borough = source.borough,
        target.state = source.state,
        target.zipcode = source.zipcode,
        target.street = source.street,
        target.country_code = source.country_code,
        target.state_code = source.state_code,
        target.borough_code = source.borough_code,
        target.neighbor_code = source.neighbor_code,
        target.district_manager = source.district_manager,
        target.src_district_id = source.src_district_id,
        target.zone_manager = source.zone_manager,
        target.src_zone_id = source.src_zone_id,
        target.zone_supervisor = source.zone_supervisor,
        target.region_name = source.region_name,
        target.src_region_id = source.src_region_id,
        target.store_type = source.store_type,
        target.first_sale_or_fallback_date = source.first_sale_or_fallback_date,
        target.store_classification = source.store_sales_category,
        target.store_status = source.store_status

WHEN NOT MATCHED THEN
    INSERT (
        src_store_id, store_name, update_date_stores, neighborhood, borough, state, 
        zipcode, street, country_code, state_code, borough_code, neighbor_code, 
        district_manager, src_district_id, zone_manager, src_zone_id, zone_supervisor, 
        region_name, src_region_id, store_type, first_sale_or_fallback_date, 
        store_classification, store_status
    )
    VALUES (
        source.src_store_id, source.store_name, source.update_date_stores, source.neighborhood, 
        source.borough, source.state, source.zipcode, source.street, source.country_code, 
        source.state_code, source.borough_code, source.neighbor_code, source.district_manager, 
        source.src_district_id, source.zone_manager, source.src_zone_id, source.zone_supervisor, 
        source.region_name, source.src_region_id, source.store_type, source.first_sale_or_fallback_date, 
        source.store_sales_category, source.store_status
    );


StatementMeta(, 3b0594a3-c742-4844-b850-37b8249fdc7f, 7, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>